# Requirement: rdkit

```bash
conda install rdkit -c rdkit
```

In [ ]:
from ipywidgets import HTML

HTML("""
<style>

.smilesApp {
    background-color: white
}
</style>
""")

In [ ]:
import nglview as nv
from IPython.display import display
from ipywidgets import Text, Button, AppLayout, VBox, HBox, Output, GridspecLayout, Checkbox, Image
import nglview as nv
from rdkit import Chem # doctest: +SKIP
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from io import BytesIO


class SmilesView(AppLayout):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.view = nv.NGLWidget()
        self.view.gui_style = 'ngl'
        self.view.background = 'black'
        self.text = Text()
        btn = Button(description='Generate')
        self.center = self.view
        self._out = Output()
        self._err = Output()
        self._check = Checkbox(description='Replace', value=True)
        bgen = VBox([self.text, self._out])
        bfull = VBox([btn])
        footer = self.footer = HBox([bgen, bfull])
        # self.left_sidebar = lbar = Image()
        # self.left_sidebar.layout.align_self = 'center'
        # lbar.layout.display = 'none'
        self.footer.layout.justify_content = 'center'
        self._m = None
        @btn.on_click
        def on_click_smiles(_):
            self._handle_smiles_changed()
            
    def _handle_smiles_changed(self):
        self._out.clear_output()
        self._err.clear_output()
        st = None
        with self._err:
            m = Chem.MolFromSmiles(self.text.value)
            print(type(m))
            m = Chem.AddHs(m)
            print(type(m))
            Chem.AllChem.EmbedMultipleConfs(m, useExpTorsionAnglePrefs=True, useBasicKnowledge=True)
            st = nv.RdkitStructure(m)
            self._m = m
            self._rdm = st
            # self.left_sidebar.value = m._repr_png_()
            # self.left_sidebar.layout.display = 'flex'

        if st:
            if self.view.n_components == 0:
                self.view.add_component(st, default=False)
                self.view.add_licorice('/0')
                self.view.center()
            else:
                struct = dict(data=st.get_structure_string(), ext='pdb')
                self.view._remote_call('replaceStructure', args=[struct])
        else:
            self._out.append_stdout("ERROR: bad input")
        
s = SmilesView()
s.add_class('smilesApp')
# smiles example:
# COc1ccc2[C@H](O)[C@@H](COc2c1)N3CCC(O)(CC3)c4ccc(F)cc4
s